In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
#%matplotlib ipympl
import glob as gb
import csv

from sklearn import manifold

In [44]:
def data_load(data_folders, res, states, file_ext="JPG", exclude=None, include=None):
    
    total = []
    total_f = []
    total_lb = []
    for df in data_folders:
        img_path = gb.glob(df+'/*.'+file_ext ) 
        
        for path in img_path:
            clb = -1
            pl = path.lower()
            
            if exclude is not None and exclude in pl:  
                continue
            
            if include is not None and include not in pl:
                continue
            

            
            for lb in states:
                if lb in pl:
                    clb = states.index(lb)
                    total_lb.append(clb)
                    break
            
            if clb == -1:
                continue
            
            img1 = cv2.imread(path, cv2.IMREAD_GRAYSCALE)#'data/Resized_256_256xBroken_0.JPG', cv2.IMREAD_GRAYSCALE)
            if img1 is None:                   #判断读入的img1是否为空，为空就继续下一轮循环
                continue
            
            res1= cv2.resize(img1,(res,res))              #对图片进行缩放，第一个参数是读入的图片，第二个是制定的缩放大小
            res1 = cv2.equalizeHist(res1)
            
            norm_image = cv2.normalize(res1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            

            
            # Fourier Transform
            f = np.fft.fft2(norm_image)
            fshift = np.fft.fftshift(f)
            fres = 20*np.ma.log(np.abs(fshift))
            fres2 = fres.flatten().tolist()   
            
            res1_1 = norm_image.flatten()/255.0 #res1_1 = res1.reshape(1,784)/255.0       #将表示图片的二维矩阵转换成一维
            #im_data = np.concatenate((im_data, res1_1))
            res1_1_1 = res1_1.tolist()                     #将numpy.narray类型的矩阵转换成list
            total.append(res1_1_1)  
            total_f.append(fres2)
            
    im_dataf = np.array(total_f)
    print(im_dataf.shape)
    im_dataf[np.isnan(im_dataf)] = np.nanmean(im_dataf)
    im_dataf[np.isinf(im_dataf)] = np.nanmean(im_dataf)

    return np.array(total), im_dataf, np.array(total_lb)


In [ ]:
# Preprocess IMG Data
fd = ['data']
res = 64
im_data, im_dataf, im_lb = data_load(fd, res, ['dripping', 'jetting', 'wetting', 'broken'], include = '30oct17')

In [19]:
im_data.tofile('data/img'+str(res)+'.bin')
im_dataf.tofile('data/imgf'+str(res)+'.bin')
im_lb.tofile('data/label.bin')

In [43]:
im_dataf[np.isinf(im_dataf)]

array([], dtype=float64)

In [38]:
im_dataf

array([[-19.74771186,  -2.55005334,  12.68803787, ...,  21.08007406,
         13.77349297, -13.87168443],
       [-29.27449045, -15.46500457,   2.27589869, ...,  10.89111676,
          8.08987534,  -1.65934216],
       [ 12.22938158, -10.94257874,  13.62892605, ...,   5.3713374 ,
         13.61930086,  15.00393494],
       ...,
       [ 12.22938973,  -9.49619248,   4.48996935, ...,   1.23855405,
          6.09867907, -13.28371595],
       [-16.99832164, -17.64197481,   7.53085686, ..., -19.67672186,
         13.19493103,  25.98498132],
       [ -7.75942885,  -3.74111103,  -8.22387637, ...,  18.07357053,
          9.67285123,  17.34275439]])